In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize':(12,6)}, style="whitegrid")

## Проект e-commerce

Продакт-менеджер Василий попросил вас проанализировать совершенные покупки и ответить на следующие вопросы:

1. Сколько у нас пользователей, которые совершили покупку только один раз?  

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? 

3. По каждому товару определить, в какой день недели товар чаще всего покупается. 

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. 

5. Напиши функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц.

6. Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R, F, M (и опиши подробно, как были построены метрики R, F, M). 


##### Файлы:

##### olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей
- customer_id — позаказный идентификатор пользователя (аналог номера паспорта)
- customer_unique_id —  уникальный идентификатор пользователя
- customer_zip_code_prefix —  почтовый индекс пользователя
- customer_city —  город доставки пользователя
- customer_state —  штат доставки пользователя
 
#### olist_orders_dataset.csv —  таблица заказов
- order_id —  уникальный идентификатор заказа (номер чека)
- customer_id —  позаказный идентификатор пользователя (аналог номера паспорта)
- order_status —  статус заказа
- order_purchase_timestamp —  время создания заказа
- order_approved_at —  время подтверждения оплаты заказа
- order_delivered_carrier_date —  время передачи заказа в логистическую службу
- order_delivered_customer_date —  время доставки заказа
- order_estimated_delivery_date —  обещанная дата доставки

#### olist_order_items_dataset.csv —  товарные позиции, входящие в заказы
- order_id —  уникальный идентификатор заказа (номер чека)
- order_item_id —  идентификатор товара внутри одного заказа
- product_id —  ид товара (аналог штрихкода)
- seller_id — ид производителя товара
- shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике
- price —  цена за единицу товара
- freight_value —  вес товара

###### Уникальные статусы заказов в таблице olist_order_items_dataset:

- created —  создан
- approved —  подтверждён
- invoiced —  выставлен счёт
- processing —  в процессе сборки заказа
- shipped —  отгружен со склада
- delivered —  доставлен пользователю
- unavailable —  недоступен
- canceled —  отменён

In [2]:
customers = pd.read_csv('olist_customers_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')

In [3]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [5]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
# перевожу столбцы с датами в формат datetime
orders[['order_purchase_timestamp', 
        'order_approved_at', 
        'order_delivered_carrier_date', 
        'order_delivered_customer_date', 
        'order_estimated_delivery_date']] = orders[['order_purchase_timestamp', 
                                                    'order_approved_at', 
                                                    'order_delivered_carrier_date', 
                                                    'order_delivered_customer_date', 
                                                    'order_estimated_delivery_date']].astype('datetime64')

In [7]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [8]:
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [9]:
items.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

#### Сколько у нас пользователей, которые совершили покупку только один раз?

In [10]:
# мерджу уникальные ид клиентов с ид и статусами заказов
df_1 = customers[['customer_id', 'customer_unique_id']]\
                .merge(orders[['order_id', 'customer_id', 'order_status']], on='customer_id')

In [11]:
df_1.head()

,customer_id,customer_unique_id,order_id,order_status
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered


In [12]:
# считаю пользователей, которые совершили покупку только один раз (покупка совершена при статусе "delivered")

df_1[df_1.order_status == 'delivered'].groupby('customer_unique_id')\
                                      .agg({'order_id' : 'count'})\
                                      .query('order_id == 1')\
                                      .count()

order_id    90557
dtype: int64

##### 90557 пользователей совершили покупку только один раз

#### Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? 

Смотрим на заказы со статусами unavailable и canceled

In [23]:
orders['month'] = orders.order_purchase_timestamp.dt.strftime('%Y-%m')

In [24]:
# Детализация по месяцам и причинам

pd.crosstab(orders.query('order_status == "canceled" or order_status == "unavailable"').month, 
            orders.query('order_status == "canceled" or order_status == "unavailable"').order_status)

order_status,canceled,unavailable
month,,
2016-09,2,0
2016-10,24,7
2017-01,3,10
2017-02,17,45
2017-03,33,32
2017-04,18,9
2017-05,29,31
2017-06,16,24
2017-07,28,52


In [25]:
# В среднем не доставлено в месяц

pd.crosstab(orders.query('order_status == "canceled" or order_status == "unavailable"').month, 
            orders.query('order_status == "canceled" or order_status == "unavailable"').order_status)\
            .mean().round(2)

order_status
canceled       26.04
unavailable    25.38
dtype: float64

##### По каждому товару определить, в какой день недели товар чаще всего покупается. 

In [16]:
orders['day_of_week'] = orders.order_purchase_timestamp.dt.day_name()

In [17]:
df_2 = items[['order_id', 'product_id']].merge(orders[['order_id', 'day_of_week']], on='order_id')

In [18]:
# вывожу id товара и день недели, в который его чаще покупали

df_2.groupby(['product_id', 'day_of_week'], as_index=False)\
    .agg({'order_id' : 'count'})\
    .sort_values('order_id', ascending=False)\
    .drop_duplicates(subset=['product_id'])\
    .reset_index(drop=True)\
    [['product_id', 'day_of_week']]

,product_id,day_of_week
0,422879e10f46682990de24d770e7f83d,Wednesday
1,99a4788cb24856965c36a24e339b6058,Monday
2,aca2eb7d00ea1a7b8ebd4e68314663af,Thursday
3,53b36df67ebb7c41585e8d54d6772e08,Tuesday
4,389d119b48cf3043d311335e499d9c6b,Thursday
...,...,...
32946,6432d844f2be41bb6f2fd279e23415da,Friday
32947,6421ae053bb80dff3ceae1f217f54ed4,Monday
32948,642247ccf62585635cefa1bf09f59b94,Sunday
32949,642e125cbc752b4eeb8faf442319c39f,Thursday


In [19]:
items.product_id.nunique()

32951

#### Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. 


In [20]:
# Добавляю столбец с количеством недель в текущем месяце
orders['weeks_in_month'] = round(orders.order_purchase_timestamp.dt.daysinmonth/7, 2)

In [21]:
df_3 = customers[['customer_id', 'customer_unique_id']]\
                .merge(orders[['order_id', 'customer_id', 'month', 'weeks_in_month']], on='customer_id')

In [22]:
df_3 = df_3.groupby(['customer_unique_id','month', 'weeks_in_month'], as_index=False)\
           .agg({'order_id' : 'count'})\
           .rename(columns={'order_id' : 'total_in_month'})\
           .reset_index(drop=True)

In [23]:
df_3 ['mean_for_week'] = df_3.total_in_month / df_3.weeks_in_month

In [24]:
# среднее количество покупок для каждого пользователя в неделю (по месяцам)
df_3[['customer_unique_id','month', 'mean_for_week']]

,customer_unique_id,month,mean_for_week
0,0000366f3b9a7992bf8c76cfdf3221e2,5,0.225734
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5,0.225734
2,0000f46a3911fa3c0805444483337064,3,0.225734
3,0000f6ccb0745a6a4b88665a16c9f078,10,0.225734
4,0004aac84e0df4da2b147fca70cf8255,11,0.233100
...,...,...,...
97996,fffcf5a5ff07b0908bd4e2dbc735a684,6,0.233100
97997,fffea47cd6d3cc0a88bd621562a9d061,12,0.225734
97998,ffff371b4d645b6ecea244b27531430a,2,0.250000
97999,ffff5962728ec6157033ef9805bacc48,5,0.225734


#### Напиши функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц. 

In [25]:
# Собираю датафрейм, с которым буду работать
df_4 = orders[['order_purchase_timestamp' , 'customer_id', 'order_id']].merge(customers[['customer_id', 'customer_unique_id']])
df_4.head()

,order_purchase_timestamp,customer_id,order_id,customer_unique_id
0,2017-10-02 10:56:33,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff
1,2018-07-24 20:41:37,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231
2,2018-08-08 08:38:49,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8
3,2017-11-18 19:28:06,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977
4,2018-02-13 21:18:39,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6


In [26]:
# Функция для разбиения на когорты по месяцам
def cohort_by_month(df):
    df = df.groupby('customer_unique_id', as_index=False).agg({'order_purchase_timestamp' : 'min'})
    df['Cohort'] = df['order_purchase_timestamp'].dt.strftime('%Y-%m')
        
    return df[['customer_unique_id', 'Cohort']]

In [27]:
cohorts = cohort_by_month(df_4)
cohorts.head()

,customer_unique_id,Cohort
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05
2,0000f46a3911fa3c0805444483337064,2017-03
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10
4,0004aac84e0df4da2b147fca70cf8255,2017-11


In [28]:
df_4 = df_4.merge(cohorts)
df_4['order_purchase_timestamp'] = df_4['order_purchase_timestamp'].dt.strftime('%Y-%m')
df_4.rename(columns={'order_purchase_timestamp': 'order_period'}, inplace=True)
df_4.head()

,order_period,customer_id,order_id,customer_unique_id,Cohort
0,2017-10,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,2017-09
1,2017-09,31f31efcb333fcbad2b1371c8cf0fa84,69923a4e07ce446644394df37a710286,7c396fd4830fd04220f754e42b4e5bff,2017-09
2,2018-07,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,2018-07
3,2018-08,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,2018-08
4,2017-11,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,2017-11


In [29]:
# Функция для создания периодов для когорт
def cohort_period(df):
    df['cohort_period'] = np.arange(len(df))
    return df

In [30]:
# Функция для построения когортного анализа

def retention_by_cohort(df):
    # считаем уникальных пользователей и число заказов по когортам и периодам заказов
    cohorts = df.groupby(['Cohort', 'order_period']).agg({'customer_unique_id': 'nunique',
                                                           'order_id': 'nunique'})
    # переименовываем колонки
    cohorts.rename(columns={'customer_unique_id': 'total_users',
                            'order_id': 'total_orders'}, inplace=True)  

    cohorts = cohorts.groupby(level=0).apply(cohort_period)
    
    # реиндексируем датафрейм
    cohorts.reset_index(inplace=True)
    cohorts.set_index(['Cohort', 'cohort_period'], inplace=True)

    # создаем серию, в которую помещаем размер когорт
    cohort_group_size = cohorts['total_users'].groupby(level=0).first()
    cohort_group_size.head()

    # считаем retention
    user_retention = cohorts['total_users'].unstack(1).divide(cohort_group_size, axis=0)

    return user_retention

In [31]:
user_retention = retention_by_cohort(df_4)

In [32]:
# добавим стиля
ur_style = (user_retention
            .style
            .set_caption('User retention by cohort')  # добавляем подпись
            .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
            .highlight_null('white')  # делаем белый фон для значений NaN
            .format("{:.2%}", na_rep=""))  # числа форматируем как проценты, NaN заменяем на пустоту
ur_style

cohort_period,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Cohort,,,,,,,,,,,,,,,,,,
2016-09,100.00%,,,,,,,,,,,,,,,,,
2016-10,100.00%,0.31%,0.31%,0.31%,0.31%,0.31%,0.31%,0.62%,0.62%,,,,,,,,,
2016-12,100.00%,100.00%,,,,,,,,,,,,,,,,
2017-01,100.00%,0.39%,0.26%,0.13%,0.39%,0.13%,0.52%,0.13%,0.13%,0.39%,0.13%,0.79%,0.39%,0.13%,0.13%,0.26%,0.39%,0.13%
2017-02,100.00%,0.23%,0.29%,0.11%,0.40%,0.11%,0.23%,0.17%,0.17%,0.23%,0.11%,0.29%,0.17%,0.17%,0.11%,0.06%,0.06%,0.23%
2017-03,100.00%,0.49%,0.38%,0.38%,0.34%,0.15%,0.15%,0.30%,0.34%,0.08%,0.38%,0.15%,0.23%,0.11%,0.15%,0.23%,0.08%,0.15%
2017-04,100.00%,0.60%,0.21%,0.17%,0.34%,0.26%,0.34%,0.30%,0.30%,0.17%,0.26%,0.09%,0.09%,0.04%,0.09%,0.09%,0.21%,
2017-05,100.00%,0.50%,0.50%,0.39%,0.31%,0.33%,0.42%,0.17%,0.25%,0.31%,0.25%,0.33%,0.25%,0.03%,0.19%,0.25%,,
2017-06,100.00%,0.48%,0.35%,0.41%,0.25%,0.38%,0.38%,0.22%,0.13%,0.22%,0.32%,0.35%,0.16%,0.13%,0.19%,,,


##### Самый высокий retention на 3-й месяц - у когорты 2017-06

#### Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R, F, M (и опиши подробно, как были построены метрики R, F, M). 

In [27]:
# собираю датафрейм, с которым буду работать
rfm_df = (customers[['customer_unique_id', 'customer_id']]
          .merge(orders[['customer_id', 'order_id', 'order_status', 'order_purchase_timestamp']])
          .merge(items[['order_id', 'product_id', 'price']]))

In [28]:
rfm_df.shape

(112650, 7)

In [29]:
rfm_df.head()

,customer_unique_id,customer_id,order_id,order_status,order_purchase_timestamp,product_id,price
0,861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,a9516a079e37a9c9c36b9b78b10169e8,124.99
1,290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,4aa6014eceb682077f9dc4bffebc05b0,289.00
2,060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,bd07b66896d6f1494f5b86251848ced7,139.94
3,259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,a5647c44af977b148e0a3a4751a09e2e,149.94
4,345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,9391a573abe00141c56e38d84d7d5b3b,230.00


###### R - recency — давность 

In [30]:
# Добавляю колонку, которая будет учитывать давность покупки в днях относительно последнего заказа в датафрейме

rfm_df['recency'] = (rfm_df.order_purchase_timestamp.max() - rfm_df.order_purchase_timestamp).dt.days

In [31]:
(rfm_df.groupby(['customer_unique_id'])
            .agg({'recency' : 'min'})
            .sort_values('recency').describe())

,recency
count,95420.000000
mean,242.600377
std,153.160320
min,0.000000
25%,118.000000
50%,223.000000
75%,352.000000
max,728.000000


В группе "recency" три категории. Мне кажется, есть смысл отсортировать пользователей, учитывая распределение давности покупок по квартилям:

 - пользователи, совершвшие последнюю покупку более, чем 352 дня назад - давние пользователи - 1
 - пользователи, совершвшие последнюю покупку 118-352 дня назад - относительно недавние, "спящие", пользователи (это около половины всех пользователей) - 2
 - пользователи, совершвшие последнюю покупку 0-117 дней назад - недавние пользователи (их примерно 25%) - 3

In [32]:
rfm_df['recency_cut'] = pd.cut(rfm_df.recency, 
                               [0, 118, 353, 730], 
                               labels=['3', '2', '1'], 
                               right=False)

In [33]:
(rfm_df.order_purchase_timestamp.max() - rfm_df.order_purchase_timestamp).dt.days

0         474
1         233
2         106
3         173
4          35
         ... 
112645    148
112646    152
112647    147
112648    303
112649    257
Name: order_purchase_timestamp, Length: 112650, dtype: int64

In [34]:
rfm_df.head()

,customer_unique_id,customer_id,order_id,order_status,order_purchase_timestamp,product_id,price,recency,recency_cut
0,861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,a9516a079e37a9c9c36b9b78b10169e8,124.99,474,1
1,290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,4aa6014eceb682077f9dc4bffebc05b0,289.00,233,2
2,060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,bd07b66896d6f1494f5b86251848ced7,139.94,106,3
3,259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,a5647c44af977b148e0a3a4751a09e2e,149.94,173,2
4,345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,9391a573abe00141c56e38d84d7d5b3b,230.00,35,3


In [35]:
rfm_df.order_purchase_timestamp.dt.day

0         16
1         12
2         19
3         13
4         29
          ..
112645     7
112646     4
112647     8
112648     3
112649    19
Name: order_purchase_timestamp, Length: 112650, dtype: int64

In [36]:
# Начиаю собрать итоговый датафрейм
# Записываю в него уникальных пользователей, последнюю активность и их максимальный ранг по шкале recency

rfm_result = (rfm_df.groupby(['customer_unique_id'], as_index=False)
                    .agg({'recency_cut' : 'max', 'recency' : 'min'}))

##### F - frequency — частота

In [37]:
# смотрю описательную статистику по количеству выполненных заказов на клиента

rfm_df[rfm_df.order_status == 'delivered'].groupby(['customer_unique_id']).agg({'order_id' : 'nunique'}).describe()

,order_id
count,93358.000000
mean,1.033420
std,0.209097
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,15.000000


In [38]:
# большинство клиентов совершает только один заказ, поэтому
# считаю, сколько клиентов сделали сколько заказов
(rfm_df[rfm_df.order_status == 'delivered'].groupby(['customer_unique_id'], as_index=False)
                                           .agg({'order_id' : 'nunique'})
                                           .groupby('order_id')
                                           .agg({'customer_unique_id' : 'count'})
)

,customer_unique_id
order_id,
1,90557
2,2573
3,181
4,28
5,9
6,5
7,3
9,1
15,1


In [39]:
(rfm_df[rfm_df.order_status != 'delivered'].groupby(['customer_unique_id'], as_index=False)
                                           .agg({'order_id' : 'nunique'})
                                           .groupby('order_id')
                                           .agg({'customer_unique_id' : 'count'})
)

,customer_unique_id
order_id,
1,2172
2,8


Группу "frequency" (частота покупок) я разобью на категории следующим образом:

- у клиента одна покупка или нет выполненных заказов - 1
- у клиента две покупки - 3
- клиент совершил 3 и более покупки - 3


In [40]:
temp = (
            rfm_df[rfm_df.order_status == 'delivered']
                    .groupby(['customer_unique_id'], as_index=False)
                    .agg({'order_id' : 'nunique'})
        )
temp['frequency'] = temp['order_id']
temp['frequency_cut'] = pd.cut(temp.order_id, [0, 1, 2, 100], labels=['1', '2', '3'])

In [41]:
temp.head()

,customer_unique_id,order_id,frequency,frequency_cut
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,1,1
2,0000f46a3911fa3c0805444483337064,1,1,1
3,0000f6ccb0745a6a4b88665a16c9f078,1,1,1
4,0004aac84e0df4da2b147fca70cf8255,1,1,1


In [42]:
# Добавляю frequency к результирующему датафрейму
rfm_result = rfm_result.merge(temp[['customer_unique_id', 'frequency', 'frequency_cut']], how='left')

In [44]:
# В датафрейме появились пустые ячейки - это frequency клиентов, у которых нет успешно завершенных заказов. Я заполню их единицами (для рангов) и нолями
rfm_result.frequency_cut.fillna('1', inplace=True)
rfm_result.frequency.fillna(0, inplace=True)

In [47]:
rfm_result.isna().sum()

customer_unique_id    0
recency_cut           0
recency               0
frequency             0
frequency_cut         0
dtype: int64

In [48]:
rfm_result.head()

,customer_unique_id,recency_cut,recency,frequency,frequency_cut
0,0000366f3b9a7992bf8c76cfdf3221e2,3,115,1.0,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2,118,1.0,1
2,0000f46a3911fa3c0805444483337064,1,541,1.0,1
3,0000f6ccb0745a6a4b88665a16c9f078,2,325,1.0,1
4,0004aac84e0df4da2b147fca70cf8255,2,292,1.0,1


##### M - monetary — деньги

In [49]:
rfm_df.head()

,customer_unique_id,customer_id,order_id,order_status,order_purchase_timestamp,product_id,price,recency,recency_cut
0,861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,a9516a079e37a9c9c36b9b78b10169e8,124.99,474,1
1,290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,4aa6014eceb682077f9dc4bffebc05b0,289.00,233,2
2,060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,bd07b66896d6f1494f5b86251848ced7,139.94,106,3
3,259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,a5647c44af977b148e0a3a4751a09e2e,149.94,173,2
4,345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,9391a573abe00141c56e38d84d7d5b3b,230.00,35,3


In [50]:
# смотрю описательную статистику по сумме покупок на клиента

(
    rfm_df[rfm_df.order_status == 'delivered']
     .groupby(['customer_unique_id'])
     .agg({'price' : 'sum'})
     .describe()
)

,price
count,93358.000000
mean,141.621480
std,215.694014
min,0.850000
25%,47.650000
50%,89.730000
75%,154.737500
max,13440.000000


Группу "monetary", как и "recency", я разделю, отталкиваясь от распределения сумм по квартилям:

- клиенты, которые потратили до 47 ед. денег и клиенты, чьи заказы не были успешно доставлены - 1
- клиенты, потратившие от 47 до 154 ед. денег - 2
- клиенты, потратившие больше 154 ед.денег - 3

In [51]:
temp = (
        rfm_df[rfm_df.order_status == 'delivered']
             .groupby(['customer_unique_id'], as_index=False)
             .agg({'price' : 'sum'})
        )
temp['monetary'] = temp.price
temp['monetary_сut'] = pd.cut(temp.price, [0, 47, 154, 100000], labels=['1', '2', '3'])

In [52]:
temp.head()

,customer_unique_id,price,monetary,monetary_сut
0,0000366f3b9a7992bf8c76cfdf3221e2,129.90,129.90,2
1,0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,18.90,1
2,0000f46a3911fa3c0805444483337064,69.00,69.00,2
3,0000f6ccb0745a6a4b88665a16c9f078,25.99,25.99,1
4,0004aac84e0df4da2b147fca70cf8255,180.00,180.00,3


In [53]:
# Добавляю frequency к результирующему датафрейму и заполняю пустые значения единицами и нолями, как в случае с frequency

rfm_result = rfm_result.merge(temp[['customer_unique_id', 'monetary', 'monetary_сut']], how='left')
rfm_result.monetary_сut.fillna('1', inplace=True)
rfm_result.monetary.fillna(0, inplace=True)
rfm_result.head()

,customer_unique_id,recency_cut,recency,frequency,frequency_cut,monetary,monetary_сut
0,0000366f3b9a7992bf8c76cfdf3221e2,3,115,1.0,1,129.90,2
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2,118,1.0,1,18.90,1
2,0000f46a3911fa3c0805444483337064,1,541,1.0,1,69.00,2
3,0000f6ccb0745a6a4b88665a16c9f078,2,325,1.0,1,25.99,1
4,0004aac84e0df4da2b147fca70cf8255,2,292,1.0,1,180.00,3


In [54]:
rfm_result.shape

(95420, 7)

In [58]:
rfm_result['RFM_cluster'] = rfm_result.recency_cut.astype(str) + rfm_result.frequency_cut.astype(str) + rfm_result.monetary_сut.astype(str)

##### Вывод среднего для каждого кластера

In [62]:
rfm_result.groupby('RFM_cluster', as_index=False)[['recency', 'frequency', 'monetary']].mean()

,RFM_cluster,recency,frequency,monetary
0,111,462.218741,0.889949,25.701659
1,112,454.205413,1.000000,89.157182
2,113,458.214141,1.000000,360.508744
3,121,403.813953,2.000000,36.031163
4,122,333.178392,2.000000,103.451658
5,123,318.275214,2.000000,344.625795
6,131,585.750000,3.250000,32.675000
7,132,291.068966,3.137931,112.074483
8,133,241.144330,3.494845,518.868144
9,211,225.156210,0.919608,26.605612
